In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time

from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')

import predict
import base
import shop_classify as sc

import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

%load_ext autoreload
%autoreload 2

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
#读取休假表
calenders = base.ReadCalendarCSV();
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();

In [5]:
%reload_ext autoreload

In [6]:
train_pay_counts = pd.read_csv("../data/train/fill_train_data.txt", sep="\t", encoding='UTF-8', index_col=0)

In [7]:
train_pay_counts.columns = train_pay_counts.columns.map(lambda str_date:datetime.datetime.strptime(str_date, "%Y-%m-%d"))

In [8]:
def calculateSingleHolidayStd(id, start_week, end_week):
    df = train_pay_counts.loc[id][np.array(range(start_week * 7, end_week * 7))].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
    
    stds = {}
    means = {}
    deviates = {}
    for i in range(1, 6):
        if i != 4:
            temp_df = df[df['holiday'] == i];
            
            stds[i] = temp_df['count'].std()
            means[i] = temp_df['count'].mean()
            deviates[i] = (np.max(temp_df['count']) - np.min(temp_df['count'])) / np.max(temp_df['count'])
            
    return stds, means, deviates;

In [9]:
def calculateHolidayStdsData(start_week, end_week):
    shop_mean_dict = {}
    shop_std_dict = {}
    shop_deviate_dict = {}
    for id in range(1,2001):
        print(id)
        shop_std_dict[id], shop_mean_dict[id], shop_deviate_dict[id] = calculateSingleHolidayStd(id, start_week, end_week)

    return shop_std_dict, shop_mean_dict, shop_deviate_dict;

In [10]:
shop_std_dict, shop_mean_dict, shop_deviate_dict = calculateHolidayStdsData(0, 3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
shop_std_df = pd.DataFrame(shop_std_dict)
shop_std_df = shop_std_df.T

shop_mean_df = pd.DataFrame(shop_mean_dict)
shop_mean_df = shop_mean_df.T


shop_deviate_df = pd.DataFrame(shop_deviate_dict)
shop_deviate_df = shop_deviate_df.T

In [12]:
shop_mean_df

,1,2,3,5
1,234.000000,253.000000,262.666667,244.000000
2,72.916667,99.333333,132.000000,115.333333
3,68.333333,77.333333,115.666667,87.666667
4,98.166667,117.333333,199.000000,198.666667
5,219.333333,126.000000,121.666667,116.666667
6,77.833333,88.333333,86.333333,97.666667
7,121.000000,132.000000,163.000000,161.333333
8,68.666667,66.333333,92.000000,79.666667
9,233.833333,241.000000,280.666667,293.666667
10,79.916667,73.000000,72.000000,90.000000


In [13]:
std_means = []
mean_stds = []
mean_gaps = []
mean_deviates = []
for id in range(1, 2001):
    std_means.append(np.mean(shop_std_df.loc[id]))
    mean_stds.append(np.std(shop_mean_df.loc[id]))
    mean_deviates.append(np.mean(shop_deviate_df.loc[id]))
    mean_gaps.append((np.max(shop_mean_df.loc[id]) - np.min(shop_mean_df.loc[id])) / np.max(shop_mean_df.loc[id]))
    print("id=%d, std_mean=%f mean_std=%f"%(id, std_means[id-1], mean_gaps[id-1]))

id=1, std_mean=17.292947 mean_std=0.109137
id=2, std_mean=27.270529 mean_std=0.447601
id=3, std_mean=6.996613 mean_std=0.409222
id=4, std_mean=17.090607 mean_std=0.506700
id=5, std_mean=97.232411 mean_std=0.468085
id=6, std_mean=24.250749 mean_std=0.203072
id=7, std_mean=28.381733 mean_std=0.257669
id=8, std_mean=16.985829 mean_std=0.278986
id=9, std_mean=32.228964 mean_std=0.203746
id=10, std_mean=13.705015 mean_std=0.200000
id=11, std_mean=12.493290 mean_std=0.446367
id=12, std_mean=18.258220 mean_std=0.353279
id=13, std_mean=19.999753 mean_std=0.151882
id=14, std_mean=18.519450 mean_std=0.219608
id=15, std_mean=16.880525 mean_std=0.355836
id=16, std_mean=9.472590 mean_std=0.271053
id=17, std_mean=7.754609 mean_std=0.271528
id=18, std_mean=16.094956 mean_std=0.264310
id=19, std_mean=17.018971 mean_std=0.297468
id=20, std_mean=20.403616 mean_std=0.287846
id=21, std_mean=11.795180 mean_std=0.427922
id=22, std_mean=20.243406 mean_std=0.185430
id=23, std_mean=64.157313 mean_std=0.654155


In [14]:
last_week_df = train_pay_counts[np.array(range(2 * 7, 3 * 7))]

In [15]:
#last_week
last_week_trend_dict = {}
for id in range(1, 2001):
    
    df = last_week_df.loc[id].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
    
    trends = []
    for date in df.index:
        trends.append((df.loc[date]['count'] - shop_mean_df.loc[id][df.loc[date]['holiday']]) / shop_mean_df.loc[id][df.loc[date]['holiday']])

    last_week_trend_dict[id] = trends

In [16]:
df = train_pay_counts.loc[1]

,1
0,251
1,283
2,274
3,252
4,228
5,220
6,252
7,223
8,206
9,220


In [61]:
#up and down
#
shop_diffs = []
for id in range(1, 2001):
    df = train_pay_counts.loc[id]
    df = df.to_frame().set_index(np.array(range(len(df)))).loc[0:20]
    up = []
    down = []
    for i in range(1, 21):
        diff = df.loc[i].values - df.loc[i - 1].values
        if diff < 0:
            down.append(diff)
        else:
            up.append(diff)
    
    #print((np.sum(down) + np.sum(up)) / np.min(df))
    #print(np.mean(df.values))
    df_mean = np.mean(df.values)
    if df_mean == 0:
        df_mean = 1
        
    shop_diffs.append(np.abs((np.sum(down) + np.sum(up)) / df_mean))
    
        

In [62]:
last_week_trend_df = pd.DataFrame(last_week_trend_dict).T

In [63]:
shop_std_mean_df = pd.DataFrame()
shop_std_mean_df['std_1'] = shop_std_df[1]
shop_std_mean_df['std_2'] = shop_std_df[2]
shop_std_mean_df['std_3'] = shop_std_df[3]
shop_std_mean_df['std_5'] = shop_std_df[5]
shop_std_mean_df['mean_std'] = mean_stds
shop_std_mean_df['mean_gap'] = mean_gaps
shop_std_mean_df['std_mean'] = std_means
shop_std_mean_df['deviate_1'] = shop_deviate_df[1]
shop_std_mean_df['deviate_2'] = shop_deviate_df[2]
shop_std_mean_df['deviate_3'] = shop_deviate_df[3]
shop_std_mean_df['deviate_5'] = shop_deviate_df[5]
shop_std_mean_df['mean_deviate'] = mean_deviates
shop_std_mean_df['trend_0'] = last_week_trend_df[0]
shop_std_mean_df['trend_1'] = last_week_trend_df[1]
shop_std_mean_df['trend_2'] = last_week_trend_df[2]
shop_std_mean_df['trend_3'] = last_week_trend_df[3]
shop_std_mean_df['trend_4'] = last_week_trend_df[4]
shop_std_mean_df['trend_5'] = last_week_trend_df[5]
shop_std_mean_df['trend_6'] = last_week_trend_df[6]
shop_std_mean_df['smooth'] = shop_diffs

In [64]:
#shop_std_mean_df = shop_std_mean_df.set_index(np.array(range(1, 2001)))

In [65]:

shop_std_mean_df.to_csv("../data/train/common/periodicity_v2.txt", sep='\t', index=True, encoding='UTF-8')

In [46]:
shop_std_mean_df[shop_std_mean_df['std_5'].isnull()]

,std_1,std_2,std_3,std_5,mean_std


In [76]:
shop_std_mean_df

,std_1,std_2,std_3,std_5,mean_std,mean_gap,std_mean,deviate_1,deviate_2,deviate_3,deviate_5,mean_deviate
1,21.878591,18.248288,10.066446,18.330303,9.812779,0.099792,17.130907,0.272085,0.120438,0.073529,0.136364,0.150604
2,32.432587,24.583192,33.060551,18.175075,20.495709,0.420799,27.062851,0.716667,0.382609,0.403727,0.269231,0.443058
3,11.009913,5.686241,3.511885,8.082904,17.937850,0.414462,7.072735,0.375000,0.134146,0.058824,0.144330,0.178075
4,17.424121,17.785762,32.186954,1.527525,45.794529,0.502513,17.231091,0.436508,0.263158,0.279476,0.015000,0.248535
5,72.488996,108.448144,104.543452,104.710713,40.370508,0.455753,97.547826,0.576052,0.994872,0.994595,0.981043,0.886640
6,10.072465,28.307832,23.158872,35.795717,7.213522,0.207881,24.333721,0.395604,0.413223,0.401786,0.510949,0.430391
7,17.866678,14.730920,22.912878,58.286648,18.672231,0.265477,28.449281,0.363014,0.184397,0.245902,0.538462,0.332944
8,14.215868,15.534907,19.924859,18.903263,10.137236,0.278986,17.144724,0.444444,0.379747,0.336538,0.356436,0.379291
9,33.045423,43.508620,26.764404,26.501572,25.942779,0.209989,32.455005,0.331010,0.306338,0.171617,0.151235,0.240050
10,20.309425,17.691806,4.582576,12.529964,7.153280,0.200000,13.778443,0.540541,0.393258,0.118421,0.242718,0.323735
